In [ ]:
import os
import pandas as pd
from openai import OpenAI

os.environ["OPENAI_API_KEY"] = "YOUR_OPENAI_API_KEY"
client = OpenAI()

In [2]:
df = pd.read_csv('../data/cleaned_data.csv', sep=';')
documents = df.to_dict(orient='records')
prompt_template = """
You emulate a user of our legal assistant application specialized in French tax law.
Formulate 5 questions this user might ask based on the provided legal record.
Make the questions specific to this record.
The record should contain the answer to the questions, and the questions should
be complete and not too short. Use as few words as possible from the record. 

The record:

type_d_article: {type_d_article}
numéro_de_l_article_ou_de_la_loi: {numéro_de_l_article_ou_de_la_loi}
description_ou_texte_complet: {description_ou_texte_complet}
mots_clés_ou_sujets_abordés: {mots_clés_ou_sujets_abordés}
date_de_publication: {date_de_publication}
source: {source}

Provide the output in parsable JSON without using code blocks:

{{'questions': ['question1', 'question2', ..., 'question5']}}

""".strip()

In [3]:
prompt = prompt_template.format(**documents[0])

In [4]:
def llm(prompt):
    response = client.chat.completions.create(
        model='gpt-4o-mini',
        messages=[{"role": "user", "content": prompt}]
    )
    
    return response.choices[0].message.content

In [10]:
import json

questions = llm(prompt)
# Remplacer les guillemets simples par des guillemets doubles
fixed_questions = questions.replace("'", '"')

try:
    parsed_questions = json.loads(fixed_questions)
    print(parsed_questions)
except json.JSONDecodeError as e:
    print(f"JSON decoding error: {e}")
    print(f"Problematic string: {fixed_questions}")


JSON decoding error: Expecting ',' delimiter: line 6 column 17 (char 309)
Problematic string: {
  "questions": [
    "Quelles mesures la Loi du 23 octobre 2018 introduit-elle pour lutter contre la fraude fiscale?",
    "Quels types de fraudes sont abordés dans la loi publiée le 25 octobre 2018?",
    "Quelle est la date de publication de la loi renforçant la lutte contre la fraude?",
    "Quel est l"objectif principal de la loi du 23 octobre 2018?",
    "D"où provient la source de la loi relative à la fraude fiscale, sociale et douanière?"
  ]
}


In [13]:
def generate_questions(doc):
    prompt = prompt_template.format(**doc)

    response = client.chat.completions.create(
        model='gpt-4o-mini',
        messages=[{"role": "user", "content": prompt}]
    )

    json_response = response.choices[0].message.content
    print(f"Raw response for doc {doc['ID']}:\n{json_response}")
    return json_response

In [14]:
questions_raw = generate_questions(doc)
cleaned_questions_raw = questions_raw.replace("'", '"')

Raw response for doc 0:
{
  "questions": [
    "Quel est l'objectif principal de la loi du 23 octobre 2018 concernant la fraude ?",
    "Quelles types de fraudes sont visées par la loi du 23 octobre 2018 ?",
    "Quand a été publiée la loi du 23 octobre 2018 ?",
    "D'où provient le texte de la loi du 23 octobre 2018 ?",
    "Quel est le type de document légal de la loi du 23 octobre 2018 ?"
  ]
}


In [26]:
from tqdm.auto import tqdm
import json

results = {}
for doc in tqdm(documents): 
    doc_id = doc['ID']
    print(f"Processing document ID: {doc_id}") 

    if doc_id in results:
        print(f"Skipping {doc_id}, already processed.")
        continue

    questions_raw = generate_questions(doc)
    print(f"Raw questions for {doc_id}: {questions_raw}") 

    try:
        questions = json.loads(questions_raw)
        results[doc_id] = questions['questions']
        print(f"Questions added for {doc_id}: {questions['questions']}") 
    except json.JSONDecodeError as e:
        print(f"JSON Error for {doc_id}: {e}")


  0%|          | 0/56 [00:00<?, ?it/s]

Processing document ID: 0
Raw response for doc 0:
{"questions":["Quelle est l'objectif principal de la loi du 23 octobre 2018 ?","Quand la loi du 23 octobre 2018 a-t-elle été publiée ?","Quels types de fraude sont ciblés par la loi du 23 octobre 2018 ?","D'où provient l'information relative à la loi du 23 octobre 2018 ?","Quel est le type de texte légal de la loi du 23 octobre 2018 ?"]}
Raw questions for 0: {"questions":["Quelle est l'objectif principal de la loi du 23 octobre 2018 ?","Quand la loi du 23 octobre 2018 a-t-elle été publiée ?","Quels types de fraude sont ciblés par la loi du 23 octobre 2018 ?","D'où provient l'information relative à la loi du 23 octobre 2018 ?","Quel est le type de texte légal de la loi du 23 octobre 2018 ?"]}
Questions added for 0: ["Quelle est l'objectif principal de la loi du 23 octobre 2018 ?", 'Quand la loi du 23 octobre 2018 a-t-elle été publiée ?', 'Quels types de fraude sont ciblés par la loi du 23 octobre 2018 ?', "D'où provient l'information rel

In [28]:
final_results = []

for doc_id, questions in results.items():
    print(f"Processing doc_id: {doc_id}")
    for q in questions:
        print(f"Adding question: {q}")
        final_results.append((doc_id, q))

Processing doc_id: 0
Adding question: Quelle est l'objectif principal de la loi du 23 octobre 2018 ?
Adding question: Quand la loi du 23 octobre 2018 a-t-elle été publiée ?
Adding question: Quels types de fraude sont ciblés par la loi du 23 octobre 2018 ?
Adding question: D'où provient l'information relative à la loi du 23 octobre 2018 ?
Adding question: Quel est le type de texte légal de la loi du 23 octobre 2018 ?
Processing doc_id: 10
Adding question: Quels types de sanctions sont généralement appliqués en cas de fraude fiscale selon la loi du 6 décembre 2013 ?
Adding question: En quelle date la loi relative à la fraude fiscale a-t-elle été publiée ?
Adding question: Quels agissements peuvent justifier une réponse pénale face à la fraude fiscale selon le texte légal ?
Adding question: Quelle est la source officielle de la loi du 6 décembre 2013 concernant la fraude fiscale ?
Adding question: Pourquoi certaines actions de fraude fiscale entraînent-elles des sanctions pénales selon le

In [29]:
final_results[0]

(0, "Quelle est l'objectif principal de la loi du 23 octobre 2018 ?")

In [30]:
df_results = pd.DataFrame(final_results, columns=['id', 'question'])

In [31]:
df_results.to_csv('../data/ground-truth-retrieval.csv', index=False)

In [32]:
!head ../data/ground-truth-retrieval.csv

id,question
0,Quelle est l'objectif principal de la loi du 23 octobre 2018 ?
0,Quand la loi du 23 octobre 2018 a-t-elle été publiée ?
0,Quels types de fraude sont ciblés par la loi du 23 octobre 2018 ?
0,D'où provient l'information relative à la loi du 23 octobre 2018 ?
0,Quel est le type de texte légal de la loi du 23 octobre 2018 ?
10,Quels types de sanctions sont généralement appliqués en cas de fraude fiscale selon la loi du 6 décembre 2013 ?
10,En quelle date la loi relative à la fraude fiscale a-t-elle été publiée ?
10,Quels agissements peuvent justifier une réponse pénale face à la fraude fiscale selon le texte légal ?
10,Quelle est la source officielle de la loi du 6 décembre 2013 concernant la fraude fiscale ?
